# HW11: Imitation Learning

> - Full Name: **Danial Parnian**
> - Student ID: **401110307**
<!--

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DeepRLCourse/Homework-5-Questions/blob/main/RL_HW11_Dyna.ipynb)
[![Open In kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/DeepRLCourse/Homework-11-Questions/main/RL_HW5_Dyna.ipynb) -->

## Overview
This assignment investigates the strengths and limitations of both reinforcement learning (PPO, A2C) and imitation learning (DAgger, GAIL) by applying them to control tasks. Students train and evaluate agents using expert demonstrations and environment feedback, gaining hands-on experience in designing and comparing learning algorithms across paradigms.


### **Exercise 1: Training PPO, A2C, and DAgger on CartPole-v1** (60 Points)

This exercise focuses on comparing reinforcement learning (PPO, A2C) and imitation learning (DAgger) techniques on the CartPole-v1 environment. An expert policy is first trained using PPO, then used to guide a DAgger agent. Separately, an A2C agent is trained directly via interaction with the environment. Performance metrics are collected to assess how each approach learns and generalizes.

---

### **Exercise 2: Training a GAIL Model Using an Expert Policy** (100 Points)

In this task, a GAIL (Generative Adversarial Imitation Learning) agent is trained to imitate a pre-trained expert policy in several environments (e.g., CartPole-v1, Pendulum-v0, BipedalWalker-v3). The training process involves configuring the environment, initializing models, and iteratively training the agent to match the expert’s behavior using adversarial learning.


In [ ]:
!pip install stable_baselines3

In [ ]:
!pip install imitation

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO, DQN
from imitation.util.util import make_vec_env
from imitation.algorithms.dagger import SimpleDAggerTrainer
from imitation.algorithms import bc
from imitation.policies.serialize import load_policy
import os
import json
import pickle
import argparse
import torch
import tempfile
import numpy as np
from stable_baselines3 import PPO,DQN,A2C
from stable_baselines3.common.evaluation import evaluate_policy
import matplotlib.pyplot as plt

# Training PPO, A2C, and DAgger on the CartPole-v1 Environment

In this section, we explore and compare multiple reinforcement learning and imitation learning approaches within the classic **CartPole-v1** environment. The algorithms used include:

- **Proximal Policy Optimization (PPO)** – a stable, on-policy RL algorithm.
- **Advantage Actor-Critic (A2C)** – another on-policy RL algorithm known for simplicity and efficiency.
- **DAgger (Dataset Aggregation)** – an imitation learning algorithm that iteratively refines a policy using expert guidance.

---

## Main Objectives

1. **Train an expert policy using PPO**.
2. **Use the PPO expert to train a new agent via DAgger**.
3. **Train a separate agent using A2C directly**.
4. **Compare performance between PPO, DAgger, and A2C**.

This setup allows us to analyze the strengths of:
- **Reinforcement learning** (PPO and A2C) that learns from environment feedback.
- **Imitation learning** (DAgger) that learns from an expert’s demonstrations and corrections.

---

### 1. PPO Expert Training (15 Points)
- A PPO agent is trained in the CartPole-v1 environment using `stable-baselines3`.
- The model is saved and later reloaded as an expert policy for imitation learning.
- Training is done in intervals and evaluated periodically to track average rewards.

In [ ]:
# Create the CartPole-v1 environment
env = gym.make("CartPole-v1")

model = PPO("MlpPolicy", env, verbose=1)
regular_rewards = []

# Training loop
total_timesteps = 10000
eval_freq = 1000

for timestep in range(0, total_timesteps, eval_freq):
    model.learn(total_timesteps=eval_freq)

    # Evaluate the model and record the reward
    reward, _ = evaluate_policy(model, env, n_eval_episodes=10)
    regular_rewards.append(reward)

    print(f"Regular Training - Step {timestep + eval_freq}: Mean Reward = {reward}")

model.save("CartPole_v1_PPO_model.zip")
env.close()

### 2. DAgger Training with the PPO Expert (25 Points)
- A `BehaviorCloning` trainer is used to initialize the DAgger process.
- A `SimpleDAggerTrainer` collects data from both the expert and the learner during training.
- The model is updated incrementally and evaluated periodically.
- Performance rewards are stored to track learning progress.

In [ ]:
# Load the PPO model manually using Stable-Baselines3's load method
expert = PPO.load("CartPole_v1_PPO_model.zip")

# Create vectorized environment for imitation learning
rng = np.random.default_rng(0)
venv = make_vec_env("CartPole-v1", n_envs=1, rng=rng)

# Initialize the Behavior Cloning (BC) trainer
bc_trainer = bc.BC(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    rng=rng,
    policy=expert.policy
)

# Record rewards during DAgger training
dagger_rewards = []

# Train using DAgger
with tempfile.TemporaryDirectory(prefix="dagger_example_") as tmpdir:
    dagger_trainer = SimpleDAggerTrainer(
        venv=venv,
        scratch_dir=tmpdir,
        expert_policy=expert,
        bc_trainer=bc_trainer,
        rng=rng
    )

    # Training loop for DAgger
    total_dagger_timesteps = 8000
    eval_freq = 1000
    for timestep in range(0, total_dagger_timesteps, eval_freq):
        dagger_trainer.train(eval_freq)

        # Evaluate the model and record the reward
        reward, _ = evaluate_policy(dagger_trainer.policy, venv, n_eval_episodes=10)
        dagger_rewards.append(reward)
        print(f"DAgger Training - Step {timestep + eval_freq}: Mean Reward = {reward}")

# Save the DAgger policy
dagger_trainer.policy.save("CartPole_v1_DAgger_model.zip")
venv.close()

In [ ]:
# Plot the rewards for Regular PPO Training and DAgger Training
plt.figure(figsize=(10, 6))
plt.plot(range(0, 10000, 1000), regular_rewards, label="PPO Regular Training", marker="o")
plt.plot(range(0, 8000, 1000), dagger_rewards, label="DAgger Training", marker="o")
plt.xlabel("Timesteps")
plt.ylabel("Mean Reward")
plt.title("Comparison of PPO and DAgger Training Rewards")
plt.legend()
plt.grid(True)
plt.show()


### 3. A2C Agent Training (20 Points)
After DAgger training, a third agent is trained using the **Advantage Actor-Critic (A2C)** algorithm.

In [ ]:
# Create the CartPole-v1 environment
env = gym.make("CartPole-v1")

# Initialize the A2C model
model = A2C("MlpPolicy", env, verbose=1)

# Record rewards during training
regular_rewards = []

# Training loop
total_timesteps = 10000
eval_freq = 1000
for timestep in range(0, total_timesteps, eval_freq):
    model.learn(total_timesteps=eval_freq)

    # Evaluate the model and record the reward
    reward, _ = evaluate_policy(model, env, n_eval_episodes=10)
    regular_rewards.append(reward)
    print(f"Regular Training - Step {timestep + eval_freq}: Mean Reward = {reward}")

# Save the model after training
model.save("CartPole_v1_A2C_model.zip")
env.close()

In [ ]:
# Load the A2C model manually using Stable-Baselines3's load method
expert = A2C.load("CartPole_v1_A2C_model.zip")

# Create vectorized environment for imitation learning
rng = np.random.default_rng(0)
venv = make_vec_env("CartPole-v1", n_envs=1, rng=rng)

# Initialize the Behavior Cloning (BC) trainer
bc_trainer = bc.BC(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    rng=rng,
    policy=expert.policy
)

# Record rewards during DAgger training
dagger_rewards = []

# Train using DAgger
with tempfile.TemporaryDirectory(prefix="dagger_example_") as tmpdir:
    dagger_trainer = SimpleDAggerTrainer(
        venv=venv,
        scratch_dir=tmpdir,
        expert_policy=expert,
        bc_trainer=bc_trainer,
        rng=rng
    )

    # Training loop for DAgger
    total_dagger_timesteps = 8000
    eval_freq = 1000
    for timestep in range(0, total_dagger_timesteps, eval_freq):
        dagger_trainer.train(eval_freq)  # Train the agent for `eval_freq` timesteps

        # Evaluate the model and record the reward
        reward, _ = evaluate_policy(dagger_trainer.policy, venv, n_eval_episodes=10)
        dagger_rewards.append(reward)
        print(f"DAgger Training - Step {timestep + eval_freq}: Mean Reward = {reward}")

# Close the environment after DAgger training
venv.close()

In [ ]:
# Plot the rewards for Regular A2C Training and DAgger Training
plt.figure(figsize=(10, 6))
plt.plot(range(0, 10000, 1000), regular_rewards, label="A2C Regular Training", marker="o")
plt.plot(range(0, 8000, 1000), dagger_rewards, label="DAgger Training", marker="o")

plt.xlabel("Timesteps")
plt.ylabel("Mean Reward")
plt.title("Comparison of A2C and DAgger Training Rewards")
plt.legend()

plt.grid(True)
plt.show()

# Training a GAIL Model Using an Expert Policy

In this section, we walk through the process of setting up and training a **Generative Adversarial Imitation Learning (GAIL)** model using a pre-trained expert in environments such as `CartPole-v1`, `Pendulum-v0`, and `BipedalWalker-v3`.

The full pipeline includes:
- Setting up directories and configurations,
- Loading and validating the environment,
- Initializing expert and GAIL models,
- Training GAIL using the expert’s behavior,
- Saving the final results and model weights.

## Summary

This pipeline implements a complete imitation learning workflow using GAIL. The process involves:
- Setting up a Gym-compatible environment,
- Loading a pre-trained expert model,
- Training a GAIL agent to mimic the expert,
- Storing the training results and model artifacts.

By following this structure, we can compare GAIL's performance to reinforcement learning approaches like PPO or A2C, and evaluate the efficiency of learning from demonstrations.

---


### **1. `setup_directories(env_name)`** (5 Points)

```python
This function handles directory setup and expert configuration:
- Creates a base checkpoint directory (`ckpts/`).
- Validates if the provided environment name is supported.
- Loads the expert model configuration from a JSON file.
- Creates a subdirectory for storing environment-specific checkpoints.

**Purpose:** Prepare the file system and load settings for the expert model.

In [ ]:
import os
import json
import pickle
import argparse
import torch
import gym
import sys
import matplotlib.pyplot as plt
import zipfile

# First, unzip the utils folder
with zipfile.ZipFile('utils.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

# Add utils directory to Python path and import necessary modules
sys.path.append('./utils')

from nets import *
from gail import *
from funcs import *

In [ ]:
def setup_directories(env_name):
    # Create checkpoint directory
    ckpt_path = "ckpts"
    if not os.path.isdir(ckpt_path):
        os.mkdir(ckpt_path)

    # Validate environment name
    if env_name not in ["CartPole-v1", "Pendulum-v0", "BipedalWalker-v3"]:
        print("The environment name is wrong!")
        return None

    # Set expert checkpoint path to utils directory
    expert_ckpt_path = "./utils"

    # Load expert config from utils/config.json
    expert_config_file = os.path.join(expert_ckpt_path, "config.json")
    if not os.path.exists(expert_config_file):
        print(f"Expert config file not found at {expert_config_file}")
        return None

    with open(expert_config_file, 'r') as f:
        expert_config = json.load(f)

    # Create environment-specific checkpoint directory
    ckpt_path = os.path.join(ckpt_path, env_name)
    if not os.path.isdir(ckpt_path):
        os.makedirs(ckpt_path)

    print(f"Setup complete - using expert config from {expert_config_file}")
    return expert_ckpt_path, expert_config, ckpt_path

### **2. `load_and_save_configs(expert_ckpt_path, ckpt_path, env_name)`** (10 Points)
- Loads a general configuration file (`config.json`) that contains training hyperparameters specific to the environment.
- Saves these configurations to the current checkpoint folder for logging and reproducibility.

**Purpose:** Fetch and store hyperparameters needed for training the GAIL model.

In [ ]:
def load_and_save_configs(expert_ckpt_path, ckpt_path, env_name):
    # Load the config.json file from utils directory
    config_file = os.path.join(expert_ckpt_path, "config.json")
    if not os.path.exists(config_file):
        print(f"Config file not found at {config_file}")
        return None

    with open(config_file, 'r') as f:
        config = json.load(f)

    # Save the config to the checkpoint directory for reproducibility
    config_save_path = os.path.join(ckpt_path, "config.json")
    with open(config_save_path, 'w') as f:
        json.dump(config, f, indent=4)

    print(f"Configuration loaded from {config_file} and saved to {config_save_path}")
    return config

### **3. `setup_environment(env_name)`** (10 Points)

- Initializes the chosen environment using OpenAI Gym.
- Extracts the dimensions of the state and action spaces.
- Checks whether the action space is discrete (e.g., CartPole) or continuous (e.g., Pendulum).

**Purpose:** Provide key environment information for model initialization.


In [ ]:
def setup_environment(env_name):
    # Create the environment
    env = gym.make(env_name)

    # Get state dimension
    state_dim = env.observation_space.shape[0]

    # Check if action space is discrete or continuous and get action dimension
    if hasattr(env.action_space, 'n'):  # Discrete action space
        action_dim = env.action_space.n
        discrete = True
    else:  # Continuous action space
        action_dim = env.action_space.shape[0]
        discrete = False

    return env, state_dim, action_dim, discrete

### **4. `setup_device()`** (+5 Points for running on GPU)
- Checks if a GPU (`cuda`) is available.
- Falls back to CPU if GPU is not detected.

**Purpose:** Ensure the training runs on the optimal hardware available.


In [ ]:
def setup_device():
    # Check if CUDA is available and set device accordingly
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device("cpu")
        print("CUDA not available. Using CPU.")

    return device

### **5. `initialize_expert(...)`** (15 Points)

- Initializes an `Expert` model using its architecture config and environment dimensions.
- Loads pre-trained policy weights (`policy.ckpt`) from the expert checkpoint.

**Purpose:** Prepare a high-performance expert model whose behavior GAIL will learn to imitate.

In [ ]:
def initialize_expert(expert_ckpt_path, state_dim, action_dim, discrete, expert_config, device, env_name):
    # Get the environment-specific config
    if env_name in expert_config:
        train_config = expert_config[env_name]
    else:
        print(f"No configuration found for environment: {env_name}")
        return None

    # Initialize the Expert model with the correct parameters
    expert = Expert(
        state_dim=state_dim,
        action_dim=action_dim,
        discrete=discrete,
        train_config=train_config
    ).to(device)

    # Load the policy checkpoint for the expert
    policy_checkpoint_path = os.path.join(expert_ckpt_path, "policy.ckpt")
    if not os.path.exists(policy_checkpoint_path):
        print(f"Expert policy checkpoint not found at {policy_checkpoint_path}")
        return None

    # Load the trained weights
    expert.pi.load_state_dict(torch.load(policy_checkpoint_path, map_location=device))
    expert.eval()  # Set to evaluation mode

    print(f"Expert model loaded from {policy_checkpoint_path}")
    return expert

### **6. `initialize_gail_model(...)`** (15 Points)
- Initializes the `GAIL` model using environment specs and configuration parameters.
- The model includes a policy, value function, and discriminator.

**Purpose:** Set up the GAIL agent that learns by interacting with the environment and distinguishing between expert and learner behavior.


In [ ]:
def initialize_gail_model(state_dim, action_dim, discrete, config, device):
    # Initialize the GAIL model with the correct parameters
    model = GAIL(
        state_dim=state_dim,
        action_dim=action_dim,
        discrete=discrete,
        train_config=config
    ).to(device)

    print(f"GAIL model initialized with state_dim={state_dim}, action_dim={action_dim}, discrete={discrete}")
    return model

### **7. `train_gail_model(model, env, expert)`** (20 Points)
- Runs the training loop for the GAIL agent.
- The GAIL model tries to imitate the expert by fooling the discriminator into thinking the learner’s actions come from the expert.

**Purpose:** Train the policy using imitation learning via adversarial training.


In [ ]:
def train_gail_model(model, env, expert):
    # Verify the training loop is functioning as expected
    results = model.train(env, expert)
    return results

### **8. `save_results_and_checkpoints(...)`** (+5 Points)
- Saves training results (e.g., episode returns) as a pickle file.
- Saves the weights of the trained components: policy, value network, and discriminator.

**Purpose:** Store the final model and results for evaluation and future use.


In [ ]:
def save_results_and_checkpoints(ckpt_path, results, model):
    # Save training results as a pickle file
    results_path = os.path.join(ckpt_path, "results.pkl")
    with open(results_path, 'wb') as f:
        pickle.dump(results, f)
    print(f"Training results saved to {results_path}")

    # Save model weights for each component
    # Save policy network
    policy_path = os.path.join(ckpt_path, "policy.ckpt")
    torch.save(model.pi.state_dict(), policy_path)

    # Save value network
    value_path = os.path.join(ckpt_path, "value.ckpt")
    torch.save(model.v.state_dict(), value_path)

    # Save discriminator network
    discriminator_path = os.path.join(ckpt_path, "discriminator.ckpt")
    torch.save(model.d.state_dict(), discriminator_path)

    print(f"Model checkpoints saved:")
    print(f"  - Policy: {policy_path}")
    print(f"  - Value: {value_path}")
    print(f"  - Discriminator: {discriminator_path}")

### 9. **`main()`** (25 Points)
- The main function orchestrates the entire GAIL training process.
- It calls all the above functions in sequence to set up directories, load configurations, initialize models, train the GAIL agent, and save results.


In [ ]:
def main(env_name):
    # Set up directories and validate environment
    expert_ckpt_path, expert_config, ckpt_path = setup_directories(env_name)
    if not expert_ckpt_path:
        return
    # Load and save configuration files
    config = load_and_save_configs(expert_ckpt_path, ckpt_path, env_name)
    # Set up environment and get dimensions
    env, state_dim, action_dim, discrete = setup_environment(env_name)
    # Set up the device (CPU or GPU)
    device = setup_device()
    # Initialize expert model
    expert = initialize_expert(expert_ckpt_path, state_dim, action_dim, discrete, expert_config, device, env_name)
    if expert is None:
        print("Failed to initialize expert model. Exiting.")
        return
    # Initialize GAIL model
    model = initialize_gail_model(state_dim, action_dim, discrete, config[env_name], device)
    # Train the GAIL model
    results = train_gail_model(model, env, expert)
    # Close the environment
    env.close()
    # Save results and model checkpoints
    save_results_and_checkpoints(ckpt_path, results, model)

### 10. **`run_gail(env_name)`** (+5 Points if using parse_args)
- This function serves as the entry point for running the GAIL training pipeline.
- It accepts an environment name as input and executes the entire GAIL training process.


In [ ]:
# Fix for newer NumPy versions
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_
if not hasattr(np, 'int0'):
    np.int0 = np.intp

# Ensure that the command-line arguments are parsed correctly
parser = argparse.ArgumentParser(description='Train GAIL model with expert policy')
parser.add_argument('--env', type=str, default='CartPole-v1',
                    choices=['CartPole-v1', 'Pendulum-v0', 'BipedalWalker-v3'],
                    help='Environment name for GAIL training')

# For Jupyter notebook, we need to handle the case where no command line args exist
try:
    args = parser.parse_args()
except SystemExit:
    # If running in Jupyter, use default arguments
    args = argparse.Namespace(env='CartPole-v1')

# Run the Main Function
# Verify that the correct environment name is passed to the main function
main(args.env)

provide a plot like below:

In [ ]:
# Load the results from the saved pickle file
results_path = os.path.join("ckpts", "CartPole-v1", "results.pkl")
with open(results_path, "rb") as f:
    results = pickle.load(f)

# Extract results from GAIL training and rename variables
expert_reward_mean, gail_reward_iterations = results

# Create the plot
plt.figure(figsize=(12, 8))

# Plot the GAIL training progress
iterations = range(1, len(gail_reward_iterations) + 1)
plt.plot(iterations, gail_reward_iterations, 'b-', linewidth=2, marker='o', markersize=6, label='GAIL Agent Reward')

# Add horizontal line for expert performance
plt.axhline(y=expert_reward_mean, color='r', linestyle='--', linewidth=2, label=f'Expert Reward (Mean: {expert_reward_mean:.2f})')

# Customize the plot
plt.xlabel('Training Iterations', fontsize=12)
plt.ylabel('Average Reward', fontsize=12)
plt.title('GAIL Training Progress vs Expert Performance', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# Add some styling
plt.tight_layout()
plt.show()

# Print summary statistics
print(f"\nTraining Summary:")
print(f"Expert Average Reward: {expert_reward_mean:.2f}")
print(f"GAIL Final Average Reward: {gail_reward_iterations[-1]:.2f}")
print(f"Performance Gap: {abs(expert_reward_mean - gail_reward_iterations[-1]):.2f}")